In [34]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import model_afolu as ma
import model_circular_economy as mc
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(sm)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python


<module 'model_afolu' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py'>

In [5]:
sa.model_attributes.dict_attributes["cat_waste_solid"].table

,category_name,cat_waste_solid,description,data_source,hyperlink,notes
0,Chemical,chemical,Industrial and chemical solid waste - includes...,NaN,NaN,See `Princeton Environmental Health and Safety...
1,Food,food,Food waste,NaN,NaN,NaN
2,Metals,metal,Metals,NaN,NaN,NaN
3,Other,other,Other waste not mentioned in any other category,NaN,NaN,NaN
4,Paper,paper,Paper products,NaN,NaN,NaN
5,Plastic,plastic,Plastic and,NaN,NaN,NaN
6,Sanitary Napkins and Diapers,sanitary_napkins,Sanitary napkins and diapers,NaN,NaN,NaN
7,Textiles,textiles,Clothing and textiles,NaN,NaN,NaN
8,Wood,wood,Wood and wood products,NaN,NaN,NaN
9,Yard,yard,Garden and yard waste,NaN,NaN,NaN


In [33]:
class CircularEconomy:

    def __init__(self, attributes: ds.ModelAttributes):

        self.model_attributes = attributes
        self.required_dimensions = self.get_required_dimensions()
        self.required_subsectors, self.required_base_subsectors = self.get_required_subsectors()
        self.required_variables, self.output_variables = self.get_ce_input_output_fields()

        ##  set some model fields to connect to the attribute tables

        # domestic solid waste model variables
        self.modvar_waso_boc = "BOC"
        self.modvar_waso_frac_dom_compo = "Composition Fraction Domestic Solid Waste"
        self.modvar_waso_frac_ind_compo = "Composition Fraction Industrial Solid Waste"
        self.modvar_waso_dem_energy_recycle = "Energy Demand to Recycle Waste"
        self.modvar_waso_dsw_pc_init = "Initial Per Capita Annual Domestic Solid Waste Generated"
        self.modvar_waso_dsw_elast = "Elasticity of Waste Produced to GDP/Capita"
        self.modvar_waso_dsw_elast = "Initial Per VA Solid Industrial Waste Generated"
        self.modvar_waso_edrf = "Virgin Production Energy Demand Reduction Factor"
        self.modvar_waso_frac_dsw_nr_burned = "Fraction of Non-Recycled Domestic Solid Waste Burned"
        self.modvar_waso_frac_dsw_nr_landfilled = "Fraction of Non-Recycled Domestic Solid Waste Landfilled"
        self.modvar_waso_frac_dsw_nr_open = "Fraction of Non-Recycled Domestic Solid Waste Open Dumps"
        self.modvar_waso_frac_dsw_recycle = "Fraction of Waste Recycled"   
        self.modvar_waso_frac_landfill_gas_captured = "Fraction of Landfill Gas Captured at Landfills"
        self.modvar_waso_k = "K"
        
        
        # economy and general variables
        self.modvar_econ_gdp = "GDP"
        self.modvar_econ_va = "Value Added"
        self.modvar_gnrl_area = "Area of Country"
        self.modvar_gnrl_occ = "National Occupation Rate"
        self.modvar_gnrl_subpop = "Population"
        self.modvar_gnrl_pop_total = "Total Population"

        ##  MISCELLANEOUS VARIABLES

        self.time_periods, self.n_time_periods = self.get_time_periods()

        # TEMP:SET TO DERIVE FROM ATTRIBUTE TABLES---
        self.landfill_gas_frac_methane = 0.5


    ##  FUNCTIONS FOR MODEL ATTRIBUTE DIMENSIONS

    def check_df_fields(self, df_ce_trajectories):
        check_fields = self.required_variables
        # check for required variables
        if not set(check_fields).issubset(df_ce_trajectories.columns):
            set_missing = list(set(check_fields) - set(df_ce_trajectories.columns))
            set_missing = sf.format_print_list(set_missing)
            raise KeyError(f"Circular Economy projection cannot proceed: The fields {set_missing} are missing.")


    def get_required_subsectors(self):
        subsectors = list(sf.subset_df(self.model_attributes.dict_attributes["abbreviation_subsector"].table, {"sector": ["Circular Economy"]})["subsector"])
        subsectors_base = subsectors.copy()
        subsectors += ["Economy", "General"]
        return subsectors, subsectors_base

    def get_required_dimensions(self):
        ## TEMPORARY - derive from attributes later
        required_doa = [self.model_attributes.dim_time_period]
        return required_doa

    def get_ce_input_output_fields(self):
        required_doa = [self.model_attributes.dim_time_period]
        required_vars, output_vars = self.model_attributes.get_input_output_fields(self.required_subsectors)
        return required_vars + self.get_required_dimensions(), output_vars

    

    

In [30]:
circ_econ = CircularEconomy(sa.model_attributes)






In [31]:
sa.model_attributes.build_varlist("Solid Waste")

['physparam_k_chemical_industrial',
 'physparam_k_food',
 'physparam_k_glass',
 'physparam_k_metal',
 'physparam_k_nappies',
 'physparam_k_other',
 'physparam_k_paper',
 'physparam_k_plastic',
 'physparam_k_rubber_leather',
 'physparam_k_textiles',
 'physparam_k_wood',
 'physparam_k_yard',
 'frac_waso_recycled_chemical_industrial',
 'frac_waso_recycled_food',
 'frac_waso_recycled_glass',
 'frac_waso_recycled_metal',
 'frac_waso_recycled_nappies',
 'frac_waso_recycled_other',
 'frac_waso_recycled_paper',
 'frac_waso_recycled_plastic',
 'frac_waso_recycled_rubber_leather',
 'frac_waso_recycled_textiles',
 'frac_waso_recycled_wood',
 'frac_waso_recycled_yard',
 'endem_waso_recycle_chemical_industrial',
 'endem_waso_recycle_food',
 'endem_waso_recycle_glass',
 'endem_waso_recycle_metal',
 'endem_waso_recycle_nappies',
 'endem_waso_recycle_other',
 'endem_waso_recycle_paper',
 'endem_waso_recycle_plastic',
 'endem_waso_recycle_rubber_leather',
 'endem_waso_recycle_textiles',
 'endem_waso_re

In [29]:
importlib.reload(sa)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python


<module 'setup_analysis' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/setup_analysis.py'>